# Recurrent Neural Netowrk
A simple implementation for a classification problem of predicting next character in a word

## Problem description

**Goal**: Predict next characters given an imcomplete word

## Setup

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout


2025-08-17 15:13:26.570525: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-17 15:13:26.649464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 15:13:29.217210: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


## Prepare train data

In [2]:
text = "show me sand the floor"     # small dataset to train

# setup the known alphabet
chars = sorted(set(text))
char2idx = {c: i for i,c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}

print(idx2char)

# convert text to numbers (the indexes defined in the alphabet)
encoded = np.array([char2idx[c] for c in text])

# create sequences (each sequence will have a window size)
seq_length = 4
X = []
y = []

for i in range(len(encoded) - seq_length):
    X.append(encoded[i:i+seq_length])
    y.append(encoded[i+seq_length])

X = np.array(X)
y = np.array(y)


{0: ' ', 1: 'a', 2: 'd', 3: 'e', 4: 'f', 5: 'h', 6: 'l', 7: 'm', 8: 'n', 9: 'o', 10: 'r', 11: 's', 12: 't', 13: 'w'}


# Build the Model

In [3]:
model = Sequential([
    keras.layers.Embedding(input_dim=len(chars), output_dim=8),      # this will convert each integer in a dense vector
    keras.layers.SimpleRNN(32, return_sequences=False),             # this is the RNN and 32 is the memory capacity
    Dense(len(chars), activation='softmax')                         # softmax will output a probability for each character
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


2025-08-17 15:13:29.577691: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [4]:
# do the training
model.fit(X, y, epochs=200, verbose=0)


## Predict

In [6]:
def predict_text(start_seq, length=18):
    seq = [char2idx[c] for c in start_seq]
    for _ in range(length):
        x = np.array([seq[-seq_length:]])   # get last seq_length chars of array
        preds = model.predict(x, verbose=0)
        next_idx = np.argmax(preds[0])  # get the highest probability character
        seq.append(next_idx)
    return ''.join(idx2char[i] for i in seq)

print(predict_text("show "))

show me sand the floor 
